<a href="https://colab.research.google.com/github/LipsaBiswas/GradioSummaryComparer/blob/main/GradioTestApp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q gradio

     |████████████████████████████████| 1.1MB 6.6MB/s 
     |████████████████████████████████| 1.9MB 21.8MB/s 
     |████████████████████████████████| 215kB 38.4MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 3.2MB 40.7MB/s 
     |████████████████████████████████| 962kB 44.6MB/s 
     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 901kB 28.4MB/s 
     |████████████████████████████████| 3.3MB 45.7MB/s 
     |████████████████████████████████| 1.2MB 6.8MB/s 


In [ ]:
# !pip install -q transformers

In [ ]:
# !pip install -q sentencepiece

In [ ]:
# !pip install -q bert-extractive-summarizer 

In [9]:
import gradio as gr
import transformers
import sentencepiece
from summarizer import Summarizer

In [10]:
bert_model=Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [13]:
# model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-pubmed")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-pubmed")

In [ ]:
# from transformers import pipeline

In [ ]:
# summarizer = pipeline("summarization")

In [14]:

pegasus_large_covid_tokenizer = AutoTokenizer.from_pretrained("mayu0007/pegasus_large_covid")

pegasus_large_covid_model = AutoModelForSeq2SeqLM.from_pretrained("mayu0007/pegasus_large_covid")


In [21]:
from transformers import   AutoModel
Bio_ClinicalBERT_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
Bio_ClinicalBERT_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
def SummarizeText(inp):  
   
    summary_generated = bert_model(inp)
#     print(summary_generated)
    
    inputs = tokenizer.encode("summarize: " + inp, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(
        inputs, 
        max_length=128, 
        min_length=10, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True)
    
    inputs_pegasus_large_covid = pegasus_large_covid_tokenizer.encode("summarize: " + inp, return_tensors="pt", max_length=1024, truncation=True)
    outputs_pegasus_large_covid = pegasus_large_covid_model.generate(
        inputs_pegasus_large_covid, 
        max_length=150, 
        min_length=50, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True)
    
    # inputs_Bio_ClinicalBERT = Bio_ClinicalBERT_tokenizer.encode("summarize: " + inp, return_tensors="pt", max_length=1024, truncation=True)
    # outputs_Bio_ClinicalBERT = Bio_ClinicalBERT_model.generate(
    #     inputs_Bio_ClinicalBERT, 
    #     max_length=100, 
    #     min_length=10, 
    #     length_penalty=2.0, 
    #     num_beams=4, 
    #     early_stopping=True)
    
    summary_generated_pegasus_pubmed = tokenizer.decode(outputs[0])
    summary_pegasus_large_covid = pegasus_large_covid_tokenizer.decode(outputs_pegasus_large_covid[0])
    # summary_Bio_ClinicalBERT = Bio_ClinicalBERT_tokenizer.decode(outputs_Bio_ClinicalBERT[0])

    return summary_generated , summary_generated_pegasus_pubmed,summary_pegasus_large_covid,"summary_Bio_ClinicalBERT"

In [34]:
a ='''
The new coronavirus severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) uses an RNA-dependent RNA polymerase (RdRp) for the replication of its genome and the transcription of its genes1-3. Here we present a cryo-electron microscopy structure of the SARS-CoV-2 RdRp in an active form that mimics the replicating enzyme. The structure comprises the viral proteins non-structural protein 12 (nsp12), nsp8 and nsp7, and more than two turns of RNA template-product duplex. The active-site cleft of nsp12 binds to the first turn of RNA and mediates RdRp activity with conserved residues. Two copies of nsp8 bind to opposite sides of the cleft and position the second turn of RNA. Long helical extensions in nsp8 protrude along exiting RNA, forming positively charged 'sliding poles'. These sliding poles can account for the known processivity of RdRp that is required for replicating the long genome of coronaviruses3. Our results enable a detailed analysis of the inhibitory mechanisms that underlie the antiviral activity of substances such as remdesivir, a drug for the treatment of coronavirus disease 2019 (COVID-19)4.
'''
SummarizeText(a)

('The new coronavirus severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) uses an RNA-dependent RNA polymerase (RdRp) for the replication of its genome and the transcription of its genes1-3. These sliding poles can account for the known processivity of RdRp that is required for replicating the long genome of coronaviruses3.',
 '<pad> viral respiratory infections are a major cause of morbidity and mortality in the developed world. <n> there are two major types of respiratory viruses : coronavirus ( cnsv ) and enterovirus ( ev ). <n> cnsv is a member of the h5n1 coronavirus family, and ev is a member of the enterovirus family. <n> both viruses replicate by homologous recombination. <n> cnsv and ev are the causative agents of severe acute respiratory syndrome ( sars ) and severe acute respiratory distress syndrome ( sards ) </s>',
 '<pad> summarize: The new coronavirus severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) uses an RNA-dependent RNA polymerase (RdRp) for the

In [37]:
# , gr.outputs.Textbox(label=" Summary-4 (emilyalsentzer/Bio_ClinicalBERT)")
iface = gr.Interface(
  fn=SummarizeText, 
  inputs=[gr.inputs.Textbox(label="Abstract (*Enter paragraphs)",lines=10, placeholder="Enter abstract text here...") ],
    
  outputs=[
           gr.outputs.Textbox(label=" Summary-1  (Bert)") ,
           gr.outputs.Textbox(label=" Summary-2 (google/pegasus-pubmed) ") ,
           gr.outputs.Textbox(label=" Summary-3 (mayu0007/pegasus_large_covid) ")],title= "Auto summarization and Summary comparision(beta)"  )
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://16661.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7867/',
 'https://16661.gradio.app')